In [1]:
import math
import numpy as np
import pandas as pd
import time

In [2]:
train_data = r'C:\Users\chaitanya\PYTHON\Project\P3 Data\counterfeit_train.csv'
test_data = r'C:\Users\chaitanya\PYTHON\Project\P3 Data\counterfeit_test.csv'

In [3]:
cct = pd.read_csv(train_data)
ccts = pd.read_csv(test_data)

In [4]:
a = list(cct.columns.values)

In [5]:
a

['Medicine_ID',
 'Counterfeit_Weight',
 'DistArea_ID',
 'Active_Since',
 'Medicine_MRP',
 'Medicine_Type',
 'SidEffect_Level',
 'Availability_rating',
 'Area_Type',
 'Area_City_Type',
 'Area_dist_level',
 'Counterfeit_Sales']

In [6]:
ccts['Counterfeit_Sales'] = pd.Series([0]* len(ccts['Medicine_ID']))

In [7]:
cct['dtype'] = pd.Series(['train']*len(cct['Medicine_ID']))

In [8]:
ccts['dtype'] = pd.Series(['test']*len(ccts['Medicine_ID']))

In [9]:
ccts = pd.DataFrame(data = ccts)

In [10]:
cct = pd.DataFrame(data = cct)

In [11]:
All_data = cct.append(ccts)

In [12]:
All_data.dtypes 

Medicine_ID             object
Counterfeit_Weight     float64
DistArea_ID             object
Active_Since             int64
Medicine_MRP           float64
Medicine_Type           object
SidEffect_Level         object
Availability_rating    float64
Area_Type               object
Area_City_Type          object
Area_dist_level         object
Counterfeit_Sales      float64
dtype                   object
dtype: object

In [13]:
All_data.isnull().sum()

Medicine_ID               0
Counterfeit_Weight     1463
DistArea_ID               0
Active_Since              0
Medicine_MRP              0
Medicine_Type             0
SidEffect_Level           0
Availability_rating       0
Area_Type                 0
Area_City_Type            0
Area_dist_level           0
Counterfeit_Sales         0
dtype                     0
dtype: int64

In [14]:
Mean_value = 0

In [15]:
All_data['Counterfeit_Weight'] = All_data['Counterfeit_Weight'].fillna(Mean_value)

In [16]:
All_data.isnull().sum()

Medicine_ID            0
Counterfeit_Weight     0
DistArea_ID            0
Active_Since           0
Medicine_MRP           0
Medicine_Type          0
SidEffect_Level        0
Availability_rating    0
Area_Type              0
Area_City_Type         0
Area_dist_level        0
Counterfeit_Sales      0
dtype                  0
dtype: int64

In [17]:
All_data['DistArea_ID'].value_counts()

Area027    935
Area013    932
Area046    930
Area035    930
Area049    930
Area045    929
Area018    928
Area017    926
Area010    555
Area019    528
Name: DistArea_ID, dtype: int64

In [18]:
for col in ['Medicine_Type','SidEffect_Level','Area_Type','Area_City_Type','Area_dist_level']:
    
    freq = All_data[col].value_counts()
    select_catgory = freq.index[freq> 1][:-1]
    
    print(col)
    
    for cat in select_catgory:
        name = col+'_'+str(cat)
        All_data[name] = (All_data[col]== cat).astype(int)
    del All_data[col] 

Medicine_Type
SidEffect_Level
Area_Type
Area_City_Type
Area_dist_level


In [19]:
All_data = All_data.drop(['DistArea_ID','Active_Since'],1)

In [20]:
cct = All_data.iloc[0:6818,]

In [21]:
cct.shape

(6818, 30)

In [22]:
ccts = All_data.iloc[6818:,]

In [23]:
cct =cct.drop(['dtype'],1)

In [24]:
ccts = ccts.drop(['dtype'],1)

In [25]:
ccts = ccts.drop(['Counterfeit_Sales'],1)

In [26]:
cct = cct.drop(['Medicine_ID'],1)

In [27]:
x = cct.drop(['Counterfeit_Sales'],1)

In [28]:
y = cct['Counterfeit_Sales']

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
training, testing = train_test_split(cct,test_size = 0.2,random_state = 2)

In [31]:
x_train = training.drop(['Counterfeit_Sales'],1)

In [32]:
y_train = training['Counterfeit_Sales']

In [33]:
x_test = testing.drop(['Counterfeit_Sales'],1)

In [34]:
y_test = testing['Counterfeit_Sales']

In [35]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge

In [36]:
lin = LinearRegression()

In [37]:
lin.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [38]:
import statsmodels.api as sm

In [39]:
model = sm.OLS(y_train, x_train).fit()

In [40]:
model_predict = model.predict(x_test)

In [41]:
final_model = sm.OLS(y, x).fit()

In [42]:
predicted = lin.predict(x_test)

In [43]:
residual = model_predict - y_test

In [44]:
MAE = abs(sum(residual))/len(y_test)

In [45]:
MAE

16.45941203413936

In [46]:
1-(MAE/1660)

0.9900846915456992

In [47]:
import statsmodels.api as sm

In [48]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [49]:
ridge = Ridge(fit_intercept=True)

In [50]:
x_train.shape[1]

27

In [51]:
ridge.fit(x_train,y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [52]:
predicted_ridge = ridge.predict(x_test)

In [53]:
residual_ridge = predicted_ridge - y_test

In [54]:
MAE = abs(sum(residual_ridge))/len(y_test)

In [55]:
MAE

17.482204834711077

In [56]:
final_ridge = ridge.fit(x,y)

In [57]:
ccts = ccts.drop(['Medicine_ID'],1)

In [58]:
final_prediction1 = final_model.predict(ccts) 

In [59]:
final_prediction = final_ridge.predict(ccts)

In [60]:
#final_prediction1

In [61]:
ccts1 = All_data.iloc[6818:,]

In [62]:
final_solution=pd.DataFrame(list(zip(ccts1['Medicine_ID'],list(final_prediction))),
                     columns=['Medicine_ID','Counterfeit_Sales'])

In [63]:
#final_solution

In [64]:
final_solution.to_csv('chaitanyap3.csv',index=False)
